In [1]:
# pytorch is just numpy on a GPU
import torch

In [19]:
# tensor is a multi-dimensional array
x = torch.Tensor([5, 3])
y = torch.Tensor([2, 1])

print(x*y)

tensor([10.,  3.])


In [20]:
x = torch.zeros([2, 5])
print(x.shape)
print(x)

torch.Size([2, 5])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [21]:
# reshape = view
y.view([1, 2])

tensor([[2., 1.]])